Set up env and imports

In [1]:
!pip install --upgrade nada-dsl
!pip install --upgrade py-nillion-client
!pip install --upgrade nillion-python-helpers

In [2]:
import asyncio
import py_nillion_client as nillion
import os

from py_nillion_client import NodeKey, UserKey
from dotenv import load_dotenv
from nillion_python_helpers import get_quote_and_pay, create_nillion_client, create_payments_config

from cosmpy.aerial.client import LedgerClient
from cosmpy.aerial.wallet import LocalWallet
from cosmpy.crypto.keypairs import PrivateKey

In [3]:
import os

def set_environment_from_string(env_string):
    # Split the string into lines
    lines = env_string.strip().split('\n')

    # Process each line
    for line in lines:
        # Split each line into key and value
        if '=' in line:
            key, value = line.split('=', 1)

            # Strip any whitespace
            key = key.strip()
            value = value.strip()

            # Set the environment variable
            os.environ[key] = value

    print("Environment variables have been set.")

env_string = """
NILLION_CLUSTER_ID=9e68173f-9c23-4acc-ba81-4f079b639964
NILLION_BOOTNODE_MULTIADDRESS=/ip4/127.0.0.1/tcp/37939/p2p/12D3KooWMvw1hEqm7EWSDEyqTb6pNetUVkepahKY6hixuAuMZfJS
NILLION_BOOTNODE_WEBSOCKET=/ip4/127.0.0.1/tcp/54936/ws/p2p/12D3KooWMvw1hEqm7EWSDEyqTb6pNetUVkepahKY6hixuAuMZfJS
NILLION_NILCHAIN_CHAIN_ID=nillion-chain-devnet
NILLION_NILCHAIN_JSON_RPC=http://127.0.0.1:48102
NILLION_NILCHAIN_REST_API=http://localhost:26650
NILLION_NILCHAIN_GRPC=localhost:26649
NILLION_NILCHAIN_PRIVATE_KEY_0=9a975f567428d054f2bf3092812e6c42f901ce07d9711bc77ee2cd81101f42c5
NILLION_NILCHAIN_PRIVATE_KEY_1=1e491133b9408b39572a29f91644873decea554224b20e2b0b923aeb860a1c18
NILLION_NILCHAIN_PRIVATE_KEY_2=980488572f235316cdb330191f8bafe4e635efbe88b3a40f5bee9bd21047c059
NILLION_NILCHAIN_PRIVATE_KEY_3=612bb5173dc60d9e91404fcc0d1f1847fb4459a7d5160d63d84e91aacbf2ab2f
NILLION_NILCHAIN_PRIVATE_KEY_4=04f5a984eeea9dce4e5e907da69c01a61568e3071b1a91cbed89225f9fd913b5
NILLION_NILCHAIN_PRIVATE_KEY_5=5f992c58921f4af83b4c6b650c4914626664cd02020577b0ada49cfa00d2c8a4
NILLION_NILCHAIN_PRIVATE_KEY_6=8f0297d3bb647eb59b95b29550b2aebbedd9be2c954b000e772efe8c9318a42d
NILLION_NILCHAIN_PRIVATE_KEY_7=c395243df9bb68dc809668efe4125f0eb017771ed8e3747b8d6860551913fecb
NILLION_NILCHAIN_PRIVATE_KEY_8=4bb5eaa799e24ae2b48545c41331921afe7e6a8dd7a850f5fbeb20a8226664ec
NILLION_NILCHAIN_PRIVATE_KEY_9=ef4b944d4fdb0077057925fe2dde365dfa2c83cf320463b14589feccd1b2b938
"""
set_environment_from_string(env_string)

Environment variables have been set.


In [4]:
cluster_id = "9e68173f-9c23-4acc-ba81-4f079b639964"
grpc_endpoint = "localhost:26649"
chain_id = "nillion-chain-devnet"
seed = "my_seed"

In [5]:
userkey = UserKey.from_seed(seed)
nodekey = NodeKey.from_seed(seed)
client = create_nillion_client(userkey, nodekey)
party_id = client.party_id
user_id = client.user_id
party_id = client.party_id
user_id = client.user_id

In [6]:
payments_config = create_payments_config(chain_id, grpc_endpoint)
payments_client = LedgerClient(payments_config)
payments_wallet = LocalWallet(
    PrivateKey(bytes.fromhex(os.getenv("NILLION_NILCHAIN_PRIVATE_KEY_0"))),
    prefix="nillion",
)

In [7]:
program_name = "double"
program_mir_path = f"binaries/double.nada.bin"

In [9]:
async def store_program():
    receipt_store_program = await get_quote_and_pay(
        client,
        nillion.Operation.store_program(program_mir_path),
        payments_wallet,
        payments_client,
        cluster_id,
    )

    # Store the program
    action_id = await client.store_program(
        cluster_id, program_name, program_mir_path, receipt_store_program
    )
    return action_id

# Use this in your Jupyter notebook cell
action_id = await store_program()
print(f"Action ID: {action_id}")

Getting quote for operation...


RuntimeError: requesting price quote: price quote query failed: cluster lookup failed: cluster not found

In [ ]:
program_id = f"{user_id}/{program_name}"

new_secret = nillion.NadaValues(
    {
        "foo": nillion.SecretInteger(2),
    }
)

party_name = "Party1"

# Set permissions for the client to compute on the program
permissions = nillion.Permissions.default_for_user(client.user_id)
permissions.add_compute_permissions({client.user_id: {program_id}})

# Pay for and store the secret in the network and print the returned store_id
receipt_store = await get_quote_and_pay(
    client,
    nillion.Operation.store_values(new_secret, ttl_days=5),
    payments_wallet,
    payments_client,
    cluster_id,
)
# Store a secret
store_id = await client.store_values(
    cluster_id, new_secret, permissions, receipt_store
)

In [28]:
compute_bindings = nillion.ProgramBindings(program_id)
compute_bindings.add_input_party(party_name, party_id)
compute_bindings.add_output_party(party_name, party_id)

computation_time_secrets = nillion.NadaValues({})

# Pay for the compute
receipt_compute = await get_quote_and_pay(
    client,
    nillion.Operation.compute(program_id, computation_time_secrets),
    payments_wallet,
    payments_client,
    cluster_id,
)

# Compute on the secret
compute_id = await client.compute(
    cluster_id,
    compute_bindings,
    [store_id],
    computation_time_secrets,
    receipt_compute,
)

# 8. Return the computation result
print(f"The computation was sent to the network. compute_id: {compute_id}")
while True:
    compute_event = await client.next_compute_event()
    if isinstance(compute_event, nillion.ComputeFinishedEvent):
        print(f"✅  Compute complete for compute_id {compute_event.uuid}")
        print(f"🖥️  The result is {compute_event.result.value}")
        break

Getting quote for operation...
Quote cost is 3 unil
Submitting payment receipt 3 unil, tx hash 7E5592AFBD8F780BF54C0D50B26D7CBE8DAE04A09F49CD04638EF4C19D40ED8A
The computation was sent to the network. compute_id: 3662f8e9-1e86-4ca8-be8c-ce3de6ecbf9a
✅  Compute complete for compute_id 3662f8e9-1e86-4ca8-be8c-ce3de6ecbf9a
🖥️  The result is {'my_output': 4}
